# KPI Card Data Extraction Notebook

This notebook replicates the data extraction process for each Level 1 KPI card in the hospital dashboard.

It demonstrates:
1. How data is retrieved from the DuckDB database
2. How KPI values are calculated
3. How benchmark data is fetched
4. How the calculation displays are generated

In [ ]:
# Import required libraries
import duckdb
import pandas as pd
import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd()))

from data.data_manager import HospitalDataManager
from utils.formatting import format_number_compact
from kpi_hierarchy_config import KPI_METADATA

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Initialize Data Manager and Select Hospital

In [ ]:
# Initialize data manager
data_manager = HospitalDataManager()

# Select a test hospital (you can change this)
test_ccn = '310001'
print(f"Testing with Hospital CCN: {test_ccn}")

# Get hospital metadata
hospital_type = data_manager.classify_hospital_type(test_ccn)
state_code = str(test_ccn)[:2]
print(f"Hospital Type: {hospital_type}")
print(f"State Code: {state_code}")

## 2. Get KPI Data for All Years

In [ ]:
# Get KPI data (5-year history)
kpi_data = data_manager.calculate_kpis(test_ccn)

print(f"\nKPI Data Shape: {kpi_data.shape}")
print(f"Years available: {sorted(kpi_data['Fiscal_Year'].unique())}")
print(f"\nColumns available ({len(kpi_data.columns)}):")
print(kpi_data.columns.tolist())

In [ ]:
# Display first few rows
print("\nKPI Data (most recent 3 years):")
kpi_data.head(3)

## 3. Level 1 KPIs Overview

In [ ]:
# Define the 6 Level 1 KPIs
LEVEL_1_KPIS = [
    'Net_Income_Margin',
    'AR_Days',
    'Operating_Expense_per_Adjusted_Discharge',
    'Medicare_CCR',
    'Bad_Debt_Charity_Pct',
    'Current_Ratio'
]

# Get metadata for each KPI
print("Level 1 KPIs Configuration:")
print("=" * 100)
for kpi_key in LEVEL_1_KPIS:
    meta = KPI_METADATA.get(kpi_key, {})
    print(f"\n{kpi_key}:")
    print(f"  Name: {meta.get('name', 'N/A')}")
    print(f"  Category: {meta.get('category', 'N/A')}")
    print(f"  Unit: {meta.get('unit', 'N/A')}")
    print(f"  Format: {meta.get('format', 'N/A')}")
    print(f"  Higher is Better: {meta.get('higher_is_better', 'N/A')}")
    print(f"  Formula: {meta.get('formula_description', 'N/A')}")

## 4. Card #1: Net Income Margin

In [ ]:
print("=" * 100)
print("CARD #1: NET INCOME MARGIN")
print("=" * 100)

# Get latest year
latest_year = kpi_data['Fiscal_Year'].max()
print(f"\nLatest Year: {latest_year}")

# Extract relevant columns
nim_data = kpi_data[['Fiscal_Year', 'Net_Income', 'Total_Revenue', 'Net_Margin_Pct']].copy()
nim_data = nim_data.sort_values('Fiscal_Year', ascending=False)

print("\nNet Income Margin - 5 Year History:")
print(nim_data)

# Get current year values
current_net_income = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Net_Income'].iloc[0]
current_revenue = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Total_Revenue'].iloc[0]
current_margin = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Net_Margin_Pct'].iloc[0]

print(f"\n{latest_year} Net Income Margin: {current_margin:.1f}%")

# Generate calculation display
income_fmt = format_number_compact(current_net_income)
revenue_fmt = format_number_compact(current_revenue)
calculation_display = f"Net income (${income_fmt}) ÷ Total revenue (${revenue_fmt})"

print(f"\nCalculation Display:")
print(f"  {calculation_display}")

# Verify calculation
verified_margin = (current_net_income / current_revenue) * 100
print(f"\nVerification:")
print(f"  Stored Margin: {current_margin:.6f}%")
print(f"  Calculated Margin: {verified_margin:.6f}%")
print(f"  Match: {abs(current_margin - verified_margin) < 0.0001}")

## 5. Card #2: Days in AR

In [ ]:
print("=" * 100)
print("CARD #2: DAYS IN AR")
print("=" * 100)

# Extract relevant columns
ar_data = kpi_data[['Fiscal_Year', 'Accounts_Receivable', 'Total_Revenue', 'AR_Days']].copy()
ar_data = ar_data.sort_values('Fiscal_Year', ascending=False)

print("\nDays in AR - 5 Year History:")
print(ar_data)

# Get current year values
current_ar = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Accounts_Receivable'].iloc[0]
current_ar_days = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['AR_Days'].iloc[0]

print(f"\n{latest_year} Days in AR: {current_ar_days:.1f} days")

# Verify calculation
verified_ar_days = current_ar / (current_revenue / 365)
print(f"\nVerification:")
print(f"  Stored AR Days: {current_ar_days:.6f}")
print(f"  Calculated AR Days: {verified_ar_days:.6f}")
print(f"  Match: {abs(current_ar_days - verified_ar_days) < 0.0001}")

## 6. Card #3: Operating Expense per Adjusted Discharge

In [ ]:
print("=" * 100)
print("CARD #3: OPERATING EXPENSE PER ADJUSTED DISCHARGE")
print("=" * 100)

# Note: This KPI requires additional data from worksheets
# For now, we'll show what's available in the main KPI table

print("\nNote: This KPI requires Adjusted Discharge data from worksheet S-3")
print("The calculation is: Total Operating Expenses ÷ Adjusted Discharges")

# Show operating expenses
oe_data = kpi_data[['Fiscal_Year', 'Total_Operating_Expenses']].copy()
oe_data = oe_data.sort_values('Fiscal_Year', ascending=False)

print("\nTotal Operating Expenses - 5 Year History:")
print(oe_data)

## 7. Card #4: Medicare CCR

In [ ]:
print("=" * 100)
print("CARD #4: MEDICARE COST-TO-CHARGE RATIO")
print("=" * 100)

print("\nNote: Medicare CCR data comes from worksheet S-10")
print("This KPI is computed separately from the main KPI table")

## 8. Card #5: Bad Debt + Charity %

In [ ]:
print("=" * 100)
print("CARD #5: BAD DEBT + CHARITY %")
print("=" * 100)

print("\nNote: Bad Debt and Charity Care data comes from worksheet G-3")
print("This KPI is computed separately from the main KPI table")

## 9. Card #6: Current Ratio

In [ ]:
print("=" * 100)
print("CARD #6: CURRENT RATIO")
print("=" * 100)

# Extract relevant columns
cr_data = kpi_data[['Fiscal_Year', 'Current_Assets', 'Current_Liabilities', 'Current_Ratio']].copy()
cr_data = cr_data.sort_values('Fiscal_Year', ascending=False)

print("\nCurrent Ratio - 5 Year History:")
print(cr_data)

# Get current year values
current_assets = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Current_Assets'].iloc[0]
current_liabilities = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Current_Liabilities'].iloc[0]
current_ratio = kpi_data[kpi_data['Fiscal_Year'] == latest_year]['Current_Ratio'].iloc[0]

print(f"\n{latest_year} Current Ratio: {current_ratio:.2f}")

# Generate calculation display
assets_fmt = format_number_compact(current_assets)
liabilities_fmt = format_number_compact(current_liabilities)
calculation_display = f"Current assets (${assets_fmt}) ÷ Current liabilities (${liabilities_fmt})"

print(f"\nCalculation Display:")
print(f"  {calculation_display}")

# Verify calculation
verified_ratio = current_assets / current_liabilities
print(f"\nVerification:")
print(f"  Stored Ratio: {current_ratio:.6f}")
print(f"  Calculated Ratio: {verified_ratio:.6f}")
print(f"  Match: {abs(current_ratio - verified_ratio) < 0.0001}")

## 10. Benchmark Data Extraction

In [ ]:
print("=" * 100)
print("BENCHMARK DATA")
print("=" * 100)

# Get all 4 benchmark levels
all_benchmarks = {
    'state_hospital_type': data_manager.get_benchmarks(test_ccn, latest_year, 'State_Hospital_Type'),
    'state': data_manager.get_benchmarks(test_ccn, latest_year, 'State'),
    'hospital_type': data_manager.get_benchmarks(test_ccn, latest_year, 'Hospital_Type'),
    'national': data_manager.get_benchmarks(test_ccn, latest_year, 'National')
}

print("\nBenchmark Levels Available:")
for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        print(f"  {level_name}: {bench_data.get('provider_count', 0)} peers")
        print(f"    Group: {bench_data.get('group_name', 'N/A')}")

## 11. Net Income Margin Benchmarks

In [ ]:
print("\n" + "=" * 100)
print("NET INCOME MARGIN BENCHMARKS")
print("=" * 100)

# Look for Net_Margin_Pct in benchmarks (this is the database column name)
db_column = 'Net_Margin_Pct'

print(f"\nCurrent Hospital Value: {current_margin:.1f}%\n")

for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        kpi_bench = bench_data.get('kpis', {}).get(db_column, {})
        if kpi_bench:
            p25 = kpi_bench.get('P25')
            median = kpi_bench.get('Median')
            p75 = kpi_bench.get('P75')
            
            print(f"{level_name.upper()}:")
            print(f"  Peers: {bench_data.get('provider_count', 0)}")
            print(f"  P25: {p25:.1f}%")
            print(f"  Median: {median:.1f}%")
            print(f"  P75: {p75:.1f}%")
            
            # Determine quartile
            if current_margin <= p25:
                quartile = "Bottom Quartile (Poor)"
            elif current_margin <= median:
                quartile = "Second Quartile (Fair)"
            elif current_margin <= p75:
                quartile = "Third Quartile (Good)"
            else:
                quartile = "Top Quartile (Excellent)"
            
            print(f"  Hospital Position: {quartile}")
            print()

## 12. Current Ratio Benchmarks

In [ ]:
print("\n" + "=" * 100)
print("CURRENT RATIO BENCHMARKS")
print("=" * 100)

db_column = 'Current_Ratio'

print(f"\nCurrent Hospital Value: {current_ratio:.2f}\n")

for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        kpi_bench = bench_data.get('kpis', {}).get(db_column, {})
        if kpi_bench:
            p25 = kpi_bench.get('P25')
            median = kpi_bench.get('Median')
            p75 = kpi_bench.get('P75')
            
            print(f"{level_name.upper()}:")
            print(f"  Peers: {bench_data.get('provider_count', 0)}")
            print(f"  P25: {p25:.2f}")
            print(f"  Median: {median:.2f}")
            print(f"  P75: {p75:.2f}")
            
            # Determine quartile
            if current_ratio <= p25:
                quartile = "Bottom Quartile (Poor)"
            elif current_ratio <= median:
                quartile = "Second Quartile (Fair)"
            elif current_ratio <= p75:
                quartile = "Third Quartile (Good)"
            else:
                quartile = "Top Quartile (Excellent)"
            
            print(f"  Hospital Position: {quartile}")
            print()

## 13. Trend Calculation

In [ ]:
print("=" * 100)
print("TREND CALCULATIONS")
print("=" * 100)

# Net Income Margin Trend
nim_values = kpi_data['Net_Margin_Pct'].values
if len(nim_values) >= 2:
    current_value = nim_values[0]  # Most recent
    previous_value = nim_values[1]  # Previous year
    
    if not pd.isna(current_value) and not pd.isna(previous_value) and previous_value != 0:
        trend_pct = ((current_value - previous_value) / abs(previous_value)) * 100
        trend_direction = 'up' if trend_pct > 0 else ('down' if trend_pct < 0 else 'flat')
        
        print(f"\nNet Income Margin:")
        print(f"  Current Year: {current_value:.1f}%")
        print(f"  Previous Year: {previous_value:.1f}%")
        print(f"  Change: {trend_pct:+.1f}% ({trend_direction})")

# Current Ratio Trend
cr_values = kpi_data['Current_Ratio'].values
if len(cr_values) >= 2:
    current_value = cr_values[0]
    previous_value = cr_values[1]
    
    if not pd.isna(current_value) and not pd.isna(previous_value) and previous_value != 0:
        trend_pct = ((current_value - previous_value) / abs(previous_value)) * 100
        trend_direction = 'up' if trend_pct > 0 else ('down' if trend_pct < 0 else 'flat')
        
        print(f"\nCurrent Ratio:")
        print(f"  Current Year: {current_value:.2f}")
        print(f"  Previous Year: {previous_value:.2f}")
        print(f"  Change: {trend_pct:+.1f}% ({trend_direction})")

## 14. Summary: Complete Card Data for Net Income Margin

In [ ]:
print("=" * 100)
print("COMPLETE CARD DATA: NET INCOME MARGIN")
print("=" * 100)

print(f"\nHospital: CCN {test_ccn}")
print(f"Hospital Type: {hospital_type}")
print(f"Latest Year: {latest_year}")

print(f"\n📊 KPI VALUE:")
print(f"  Net Income Margin: {current_margin:.1f}%")

print(f"\n📈 TREND:")
if len(nim_values) >= 2 and not pd.isna(nim_values[0]) and not pd.isna(nim_values[1]):
    trend_pct = ((nim_values[0] - nim_values[1]) / abs(nim_values[1])) * 100
    arrow = "↑" if trend_pct > 0 else "↓"
    print(f"  {arrow} {abs(trend_pct):.1f}% vs Last Year")

print(f"\n🧮 CALCULATION:")
print(f"  {calculation_display}")

print(f"\n📅 5-YEAR HISTORY:")
for idx, row in nim_data.iterrows():
    year = int(row['Fiscal_Year'])
    value = row['Net_Margin_Pct']
    print(f"  {year}: {value:.1f}%")

print(f"\n📊 BENCHMARKS:")
for level_name, bench_data in all_benchmarks.items():
    if bench_data:
        kpi_bench = bench_data.get('kpis', {}).get('Net_Margin_Pct', {})
        if kpi_bench:
            p25 = kpi_bench.get('P25')
            median = kpi_bench.get('Median')
            p75 = kpi_bench.get('P75')
            
            # Determine quartile
            if current_margin <= p25:
                position = "Poor ⚠️"
            elif current_margin <= median:
                position = "Fair ⚡"
            elif current_margin <= p75:
                position = "Good ⭐"
            else:
                position = "Excellent ✅"
            
            print(f"  {level_name}: P25={p25:.1f}%, Median={median:.1f}%, P75={p75:.1f}% → {position}")

print("\n" + "=" * 100)

## 15. Test Different Hospitals

In [ ]:
# You can change this to test other hospitals
test_hospitals = ['310001', '310005', '310010']

print("Comparison across multiple hospitals:")
print("=" * 100)

comparison_data = []
for ccn in test_hospitals:
    try:
        kpi_data_temp = data_manager.calculate_kpis(ccn)
        if not kpi_data_temp.empty:
            latest = kpi_data_temp['Fiscal_Year'].max()
            latest_data = kpi_data_temp[kpi_data_temp['Fiscal_Year'] == latest].iloc[0]
            
            comparison_data.append({
                'CCN': ccn,
                'Year': int(latest),
                'Net_Income_Margin': f"{latest_data['Net_Margin_Pct']:.1f}%",
                'Current_Ratio': f"{latest_data['Current_Ratio']:.2f}",
                'AR_Days': f"{latest_data['AR_Days']:.1f}"
            })
    except Exception as e:
        print(f"Error loading CCN {ccn}: {e}")

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))